# Create real_layers.csv file for Ekstrom

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.io
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy.interpolate import UnivariateSpline
from matplotlib.ticker import FormatStrFormatter

from tueplots import figsizes
from sbi_ice.utils import plotting_utils,misc,modelling_utils,noise_model
data_dir,output_dir = misc.get_data_output_dirs()
color_opts = plotting_utils.setup_plots()


In [ ]:
#Load radar data product
print(Path(data_dir,"Ekstrom","IRH_data","Ekstroem_flowline_GPR_IRH_data.mat"))
mat = scipy.io.loadmat(Path(data_dir,"Ekstrom","IRH_data","Ekstroem_flowline_GPR_IRH_data.mat"))
shelf_mask = scipy.io.loadmat(Path(data_dir,"Ekstrom","IRH_data","LayerData_flowline_mask.mat"))['save_in'].astype(bool).flatten()


In [ ]:
# Spline interpolate coordinates to get correct distance measurements along path.

coordinates = np.array([(x, y) for x,y in zip(mat['psX'].T,mat['psY'].T)])[:,:,0]
print(coordinates.shape)
coordinates = coordinates[shelf_mask]
print(coordinates.shape)
coordinates = modelling_utils.trunc(coordinates,0)
dist = np.concatenate((np.array([0.0]),np.cumsum(np.sqrt((coordinates[1:,0]-coordinates[:-1,0])**2 + (coordinates[1:,1]-coordinates[:-1,1])**2))))

smoothing_constant = 5.0e5
npoints = dist.size
print("number of points: ",npoints)
print("smoothing constant lower bound: ",npoints - np.sqrt(2*npoints))
print("smoothing constant upper bound: ",npoints + np.sqrt(2*npoints))


alpha = np.linspace(0, dist[-1],dist.size)
#splines = [UnivariateSpline(dist, coords, s=smoothing_constant) for coords in coordinates.T]
splines = [UnivariateSpline(dist, coords, w = 0.05*np.ones(coords.size),s=npoints) for coords in coordinates.T]
interpolated_points = np.vstack([spl(alpha) for spl in splines]).T


new_dist = np.cumsum( np.sqrt(np.sum( np.diff(interpolated_points, axis=0)**2, axis=1 )) )
new_dist = np.insert(new_dist, 0, 0)

print(dist[-1])
print(new_dist[-1])
dist = new_dist.copy()



coordinates = interpolated_points.copy()


In [ ]:
#Calculate elevation from traveltime (we don't use depth-density relation here, to match with radar product).

eps_i = 3.15
eps = (np.sqrt(3)*eps_i)**3
c_light = 3.0e8
c_ice = c_light/np.sqrt(eps)
z = np.linspace(0.0,1000.0,5504)
rho = 917.0

def ref_ind_const_density(rho):
    return ((1/noise_model.rho_ice)*rho*(np.cbrt(3.15)-1)+1)**3
def twt2depth_const_density(twt,z,rho):
    v = c_light/(np.sqrt(ref_ind_const_density(rho)))

    dts = np.concatenate((np.array([0]),np.diff(z)))/v
    traveltimedepth = np.cumsum(dts)
    depth = np.zeros_like(twt)
    for i in range(0,twt.shape[0]):
        depth[i] = np.nan if not twt[i]>0 else z[np.argmin(np.abs(traveltimedepth-twt[i]/2))]
    return depth


sur_elevation = mat['elev_sur_bedmachine'].flatten()[shelf_mask]
layer_twt = mat['twt_layers'][:,shelf_mask]/1e9
layer_depth = np.zeros_like(layer_twt)
for i in range(layer_depth.shape[0]):
    layer_depth[i] = twt2depth_const_density(layer_twt[i],z,rho)


l2_depth = mat["depth_layers"][:,shelf_mask]
#Sanity check
print(layer_depth)
print(l2_depth)


mat_dist = mat['distance'].flatten()[shelf_mask]
print(mat_dist-dist)


In [ ]:
#Plot layer elevations

fig,axs = plt.subplots(figsize=(15,10))

dist_km = dist/1000
axs.plot(dist_km,sur_elevation,color='tab:blue')
print(layer_depth.shape[0])
axs.set_xlabel("Distance from GL [km]")
axs.set_ylabel("Elevation [m.a.s.l]")
for i in range(layer_depth.shape[0]):
        axs.plot(dist_km,sur_elevation-layer_depth[i],'r')
        axs.plot(dist_km,sur_elevation - l2_depth[i],'g--')
        print(np.mean(layer_depth[i][~np.isnan(layer_depth[i])]))



In [ ]:
#save layer depths
df = pd.DataFrame(dist,columns = ['x_coord'])
for i in range(layer_depth.shape[0]):
    df['layer {}'.format(i+1)] = l2_depth[i,:]
output_file = Path(output_dir,"Ekstrom","real_layers.csv")
df.to_csv(output_file)